<a href="https://colab.research.google.com/github/minionpika/Secret-Message-Decode/blob/main/decode_secret_msg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppose you have a Google Doc URL, which is public. You need to take the URL as an input. In it, there is a table that includes X coordinate, Character, Y coordinate. If you organise those characters into their corresponding (x,y) coordinates in a grid/matrix, it will decode a message either one character or several characters.


---
Specifications:


*   All decoded characters will be UPPER CASE.
*   There is no specific number of characters.
*   The secret message can be rotated, so while showing output, make it correctly oriented.

---
Examples: URL 1 is decoded as 'F', you need to figure out the secret message in URL 2.
1.   [doc url 1](https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub)
2.   [doc url 2](https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub)


In [ ]:
!pip install python-docx
!pip install google-api-python-client
!pip install google-auth-oauthlib python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.5 MB/s eta 0:00:00


In [1]:
import requests
from bs4 import BeautifulSoup

def fetch_published_doc_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.get_text(separator='\n')
        return content
    else:
        print(f"Failed to fetch the document. Status code: {response.status_code}")
        return None

def extract_data_after_keyword(content, keyword):
    lines = content.split('\n')
    data_map = {}
    found_keyword = False
    buffer = []

    for line in lines:
        if keyword in line:
            found_keyword = True
            buffer = []
            continue

        if found_keyword:
            buffer.append(line.strip())

            # If we've collected 3 lines, add them as a tuple to the map
            if len(buffer) == 3:
                data_map[len(data_map) + 1] = tuple(buffer)
                buffer = []  # Reset the buffer to collect next set of 3 lines

    return data_map

def process_data_map(data_map):
    position_and_message = []

    for key, (first, second, third) in data_map.items():
        position = (first, third)  # (x, y)
        secret_msg = second
        position_and_message.append((position, secret_msg))

    return position_and_message

def create_and_fill_grid(position_and_message):
    # Determine the size of the grid based on the max x and y values
    max_x = max(int(pos[0]) for pos, _ in position_and_message)
    max_y = max(int(pos[1]) for pos, _ in position_and_message)

    # Create an empty grid
    grid = [["" for _ in range(max_y+1)] for _ in range(max_x+1)]

    # Fill the grid with messages
    for (x, y), message in position_and_message:

            row_idx = int(x)   # Adjusting for 0-based indexing (x-coordinate)
            col_idx = int(y) # Adjusting for 0-based indexing (y-coordinate)

            # Place the message in the grid
            grid[row_idx][col_idx] = message


    return grid

def rotate_grid_90_deg_anticlockwise(grid):
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    # Create a new grid for the rotated matrix
    rotated_grid = [["" for _ in range(rows)] for _ in range(cols)]

    for r in range(rows):
        for c in range(cols):
            rotated_grid[cols - c - 1][r] = grid[r][c]

    return rotated_grid

if __name__ == '__main__':
    #doc_url = 'https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub'
    doc_url = 'https://docs.google.com/document/d/e/2PACX-1vSHesOf9hv2sPOntssYrEdubmMQm8lwjfwv6NPjjmIRYs_FOYXtqrYgjh85jBUebK9swPXh_a5TJ5Kl/pub'
    content = fetch_published_doc_content(doc_url)

    if content:
        keyword = 'y-coordinate'
        data_map = extract_data_after_keyword(content, keyword)
        position_and_message = process_data_map(data_map)

        # Create and fill the grid based on the extracted positions and messages
        grid = create_and_fill_grid(position_and_message)

        # Rotate the grid 90 degrees counterclockwise
        rotated_grid = rotate_grid_90_deg_anticlockwise(grid)

        # Print the rotated grid
        for row in grid:
            print(row)

# N.B - I have used grid instead of rotated_grid to decode the letters. Because it seems easier to decode with my eyes.

['█', '█', '█', '█', '█', '█', '█']
['█', '█', '█', '█', '█', '█', '█']
['█', '░', '░', '█', '░', '░', '█']
['█', '', '', '█', '', '', '█']
['█', '', '', '█', '', '', '█']
['█', '', '', '█', '', '', '█']
['█', '', '', '█', '', '', '█']
['█', '', '', '█', '', '', '█']
['█', '', '', '░', '', '', '█']
['█', '', '', '', '', '', '█']
['░', '', '', '', '', '', '░']
['', '', '', '', '', '', '']
['█', '', '', '', '', '', '█']
['█', '', '', '', '', '', '█']
['█', '█', '█', '█', '█', '█', '█']
['█', '█', '█', '█', '█', '█', '█']
['█', '░', '░', '░', '░', '░', '█']
['█', '', '', '', '', '', '█']
['░', '', '', '', '', '', '░']
['', '', '', '', '', '', '']
['', '', '█', '█', '█', '', '']
['', '█', '█', '█', '█', '█', '']
['', '█', '█', '░', '█', '█', '']
['█', '█', '░', '', '░', '█', '█']
['█', '░', '', '', '', '░', '█']
['█', '', '', '', '', '', '█']
['█', '', '', '', '', '', '█']
['█', '', '', '', '', '', '█']
['█', '', '', '', '', '', '█']
['█', '█', '', '', '', '█', '█']
['░', '█', '', '', '', 